In [ ]:
# Author: HZQ, ZTR
# Last modified: 2019/4/3

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools 
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null 
!apt-get update -qq 2>&1 > /dev/null 
!apt-get -y install -qq google-drive-ocamlfuse fuse 
from google.colab import auth 
auth.authenticate_user() 
from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default() 
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL 
vcode = getpass.getpass() 
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
!mkdir -p driver
!google-drive-ocamlfuse driver
import os

os.chdir("driver/")
!ls

In [ ]:
!pip install catboost
!pip install plotly

In [1]:
import gc
import os
import string
import time
import random
import warnings 
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
# from plotly import tools
import plotly.offline as py
# import plotly.graph_objs as go

# from scipy.stats import norm
# from scipy import stats
# from sklearn.preprocessing import StandardScaler
# from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold, KFold

import xgboost as xgb
import lightgbm as lgb
import catboost as cb


pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
color = sns.color_palette()
%matplotlib inline
py.init_notebook_mode(connected=True)


data_root = '../../santander-customer-transaction-prediction'

In [2]:
train_df = pd.read_csv(os.path.join(data_root, 'train.csv'))
test_df = pd.read_csv(os.path.join(data_root, 'test.csv'))

feature_names = [c for c in train_df.columns if c not in ['ID_code', 'target']]
X = train_df[feature_names].values.astype(np.float32)
y = train_df['target'].values
T = test_df[feature_names].values.astype(np.float32)

In [3]:
# Taking a look at how many rows and columns the train dataset contains
rows1 = train_df.shape[0]; rows2 = test_df.shape[0]
columns1 = train_df.shape[1]; columns2 = test_df.shape[1]
print("The train dataset contains {0} rows and {1} columns".format(rows1, columns1))
print("The test dataset contains {0} rows and {1} columns".format(rows2, columns2))

The train dataset contains 200000 rows and 202 columns
The test dataset contains 200000 rows and 201 columns


In [4]:
"""
XGBoost model
"""
xgb_params = {'num_round': 10000,
              'max_depth': 2,
              'colsample_bytree': 0.3,
              'learning_rate': 0.02,
              'objective': 'binary:logistic',
              'verbose': True,
              'early_stopping_rounds': 500
             }

xgboost_model = xgb.XGBClassifier(**xgb_params)

In [5]:
"""
LightGBM model
"""
# https://www.kaggle.com/fayzur/customer-transaction-prediction-strong-baseline
# Thanks fayzur. Nice Parameter
lgb_params = {'bagging_freq': 5,
              'bagging_fraction': 0.4,
              'boost_from_average':'false',
              'boost': 'gbdt',
              'feature_fraction': 0.05,
              'learning_rate': 0.01,
              'max_depth': -1,  
              'metric':'auc',
              'min_data_in_leaf': 80,
              'min_sum_hessian_in_leaf': 10.0,
              'num_leaves': 13,
              'num_threads': 8,
              'tree_learner': 'serial',
              'objective': 'binary', 
              'verbosity': 1,
              'num_boost_round': 10000,
              'early_stopping_rounds': 500
             }

stacker_params = {'bagging_freq': 5,
                  'bagging_fraction': 0.4,
                  'boost_from_average':'false',
                  'boost': 'gbdt',
                  'feature_fraction': 0.05,
                  'learning_rate': 0.01,
                  'max_depth': -1,  
                  'metric':'auc',
                  'min_data_in_leaf': 80,
                  'min_sum_hessian_in_leaf': 10.0,
                  'num_leaves': 13,
                  'num_threads': 8,
                  'tree_learner': 'serial',
                  'objective': 'binary', 
                  'verbosity': 1,
                  'num_boost_round': 10000,
                 }

lightgbm_model = lgb.LGBMClassifier(**lgb_params)
lightgbm_stacker_model = lgb.LGBMClassifier(**stacker_params)

In [15]:
"""
CatBoost model
"""
# https://www.kaggle.com/wakamezake/starter-code-catboost-baseline
cb_params = {'loss_function': 'Logloss',
             'eval_metric': 'AUC',
             'learning_rate': 0.01,
             'iterations': 10000,
             'random_seed': 42,
             'od_type': 'Iter',
             'depth': 10,
             'early_stopping_rounds': 500
            }

catboost_model = cb.CatBoostClassifier(**cb_params)

In [18]:
def train_predict(name, classifier, X, y, T, n_folds=5):
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=2019)
    pred = np.zeros(T.shape[0])
    
    for i, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train = X[train_idx]
        y_train = y[train_idx]
        X_eval = X[test_idx]
        y_eval = y[test_idx]
        
        if name == 'xgboost':
            data_eval = (X_eval, y_eval)
            classifier.fit(X_train, y_train, eval_set=data_eval)
        elif name == 'lightgbm':
            data_eval = lgb.Dataset(X_eval, label=y_eval)
            classifier.fit(X_train, y_train, eval_set=data_eval)
        elif name == 'catboost':
            data_train = cb.Pool(X_train, label=y_train)
            data_eval = cb.Pool(X_eval, label=y_eval)
            classifier.fit(data_train, eval_set=data_eval, use_best_model=True)
        
        pred += classifier.predict(T)[:]
    
    pred /= n_folds
    
    return pred

In [ ]:
class Ensemble(object):
    
    def __init__(self, stacker, base_models, model_names, n_folds=5):
        super().__init__()
        self.stacker = stacker
        self.base_models = base_models
        self.model_names = model_names
        self.n_folds = n_folds

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)
        
        skf = StratifiedKFold(n_splits=self.n_folds, shuffle=True, random_state=2019)

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))

        for i, (clf, name) in enumerate(zip(self.base_models, self.model_names)):
            S_test_i = np.zeros((T.shape[0], self.n_folds))

            for j, (train_idx, test_idx) in enumerate(skf.split(X, y)):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                y_holdout = y[test_idx]
                
                if name == 'xgboost':
                    data_eval = (X_holdout, y_holdout)
                    clf.fit(X_train, y_train, eval_set=data_eval)
                elif name == 'lightgbm':
                    data_eval = lgb.Dataset(X_holdout, label=y_holdout)
                    clf.fit(X_train, y_train, eval_set=data_eval)
                elif name == 'catboost':
                    data_train = cb.Pool(X_train, label=y_train)
                    data_eval = cb.Pool(X_eval, label=y_eval)
                    clf.fit(data_train, eval_set=data_eval, use_best_model=True)
                    
                y_pred = clf.predict(X_holdout)[:]
                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict(T)[:]

            S_test[:, i] = S_test_i.mean(1)

        self.stacker.fit(S_train, y)
        y_pred = self.stacker.predict(S_test)[:]
        
        return y_pred

In [9]:
# XGBoost
name = 'xgboost'
pred = train_predict(name, xgboost_model, X, y, T, n_folds=5)

/home/ztr/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/home/ztr/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/home/ztr/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/home/ztr/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in futu

In [11]:
# LightGBM
name = 'lightgbm'
pred = train_predict(name, lightgbm_model, X, y, T, n_folds=5)

/home/ztr/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/home/ztr/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/home/ztr/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/home/ztr/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in futu

In [ ]:
# CatBoost
name = 'catboost'
pred = train_predict(name, catboost_model, X, y, T, n_folds=5)

0:	test: 0.5778320	best: 0.5778320 (0)	total: 557ms	remaining: 1h 32m 51s
1:	test: 0.5996707	best: 0.5996707 (1)	total: 1.09s	remaining: 1h 30m 54s
2:	test: 0.6093745	best: 0.6093745 (2)	total: 1.64s	remaining: 1h 30m 54s
3:	test: 0.6380264	best: 0.6380264 (3)	total: 2.18s	remaining: 1h 30m 49s
4:	test: 0.6393291	best: 0.6393291 (4)	total: 2.69s	remaining: 1h 29m 45s
5:	test: 0.6406691	best: 0.6406691 (5)	total: 3.26s	remaining: 1h 30m 23s
6:	test: 0.6632722	best: 0.6632722 (6)	total: 3.83s	remaining: 1h 31m 10s
7:	test: 0.6664668	best: 0.6664668 (7)	total: 4.3s	remaining: 1h 29m 33s
8:	test: 0.6722527	best: 0.6722527 (8)	total: 4.93s	remaining: 1h 31m 17s
9:	test: 0.6920896	best: 0.6920896 (9)	total: 5.55s	remaining: 1h 32m 29s
10:	test: 0.6994906	best: 0.6994906 (10)	total: 6.02s	remaining: 1h 31m 8s
11:	test: 0.7075643	best: 0.7075643 (11)	total: 6.56s	remaining: 1h 31m 3s
12:	test: 0.7185357	best: 0.7185357 (12)	total: 7.1s	remaining: 1h 30m 54s
13:	test: 0.7209648	best: 0.7209648 

109:	test: 0.8139489	best: 0.8141286 (108)	total: 56.4s	remaining: 1h 24m 27s
110:	test: 0.8150844	best: 0.8150844 (110)	total: 56.8s	remaining: 1h 24m 19s
111:	test: 0.8155574	best: 0.8155574 (111)	total: 57.2s	remaining: 1h 24m 10s
112:	test: 0.8153249	best: 0.8155574 (111)	total: 57.7s	remaining: 1h 24m 4s
113:	test: 0.8163427	best: 0.8163427 (113)	total: 58.1s	remaining: 1h 23m 56s
114:	test: 0.8159656	best: 0.8163427 (113)	total: 58.5s	remaining: 1h 23m 48s
115:	test: 0.8164348	best: 0.8164348 (115)	total: 58.9s	remaining: 1h 23m 40s
116:	test: 0.8162960	best: 0.8164348 (115)	total: 59.4s	remaining: 1h 23m 34s
117:	test: 0.8164941	best: 0.8164941 (117)	total: 59.8s	remaining: 1h 23m 26s
118:	test: 0.8168771	best: 0.8168771 (118)	total: 1m	remaining: 1h 23m 20s
119:	test: 0.8174096	best: 0.8174096 (119)	total: 1m	remaining: 1h 23m 12s
120:	test: 0.8183013	best: 0.8183013 (120)	total: 1m 1s	remaining: 1h 23m 3s
121:	test: 0.8185811	best: 0.8185811 (121)	total: 1m 1s	remaining: 1h 23

214:	test: 0.8285758	best: 0.8285758 (214)	total: 1m 43s	remaining: 1h 18m 14s
215:	test: 0.8287127	best: 0.8287127 (215)	total: 1m 43s	remaining: 1h 18m 11s
216:	test: 0.8289279	best: 0.8289279 (216)	total: 1m 44s	remaining: 1h 18m 9s
217:	test: 0.8290860	best: 0.8290860 (217)	total: 1m 44s	remaining: 1h 18m 8s
218:	test: 0.8294279	best: 0.8294279 (218)	total: 1m 44s	remaining: 1h 18m 7s
219:	test: 0.8294582	best: 0.8294582 (219)	total: 1m 45s	remaining: 1h 18m 4s
220:	test: 0.8295208	best: 0.8295208 (220)	total: 1m 45s	remaining: 1h 18m 4s
221:	test: 0.8295597	best: 0.8295597 (221)	total: 1m 46s	remaining: 1h 18m 2s
222:	test: 0.8294790	best: 0.8295597 (221)	total: 1m 46s	remaining: 1h 18m 1s
223:	test: 0.8295207	best: 0.8295597 (221)	total: 1m 47s	remaining: 1h 17m 58s
224:	test: 0.8297066	best: 0.8297066 (224)	total: 1m 47s	remaining: 1h 17m 56s
225:	test: 0.8297010	best: 0.8297066 (224)	total: 1m 48s	remaining: 1h 17m 54s
226:	test: 0.8298397	best: 0.8298397 (226)	total: 1m 48s	re

319:	test: 0.8399490	best: 0.8399490 (319)	total: 2m 27s	remaining: 1h 14m 19s
320:	test: 0.8401071	best: 0.8401071 (320)	total: 2m 27s	remaining: 1h 14m 18s
321:	test: 0.8403034	best: 0.8403034 (321)	total: 2m 28s	remaining: 1h 14m 16s
322:	test: 0.8402522	best: 0.8403034 (321)	total: 2m 28s	remaining: 1h 14m 13s
323:	test: 0.8404362	best: 0.8404362 (323)	total: 2m 29s	remaining: 1h 14m 12s
324:	test: 0.8405105	best: 0.8405105 (324)	total: 2m 29s	remaining: 1h 14m 11s
325:	test: 0.8405995	best: 0.8405995 (325)	total: 2m 29s	remaining: 1h 14m 8s
326:	test: 0.8406946	best: 0.8406946 (326)	total: 2m 30s	remaining: 1h 14m 6s
327:	test: 0.8406848	best: 0.8406946 (326)	total: 2m 30s	remaining: 1h 14m 4s
328:	test: 0.8408152	best: 0.8408152 (328)	total: 2m 31s	remaining: 1h 14m 2s
329:	test: 0.8408521	best: 0.8408521 (329)	total: 2m 31s	remaining: 1h 14m 1s
330:	test: 0.8409410	best: 0.8409410 (330)	total: 2m 31s	remaining: 1h 13m 59s
331:	test: 0.8409577	best: 0.8409577 (331)	total: 2m 32s	

424:	test: 0.8489157	best: 0.8489157 (424)	total: 3m 11s	remaining: 1h 11m 46s
425:	test: 0.8489315	best: 0.8489315 (425)	total: 3m 11s	remaining: 1h 11m 45s
426:	test: 0.8489900	best: 0.8489900 (426)	total: 3m 11s	remaining: 1h 11m 44s
427:	test: 0.8490590	best: 0.8490590 (427)	total: 3m 12s	remaining: 1h 11m 43s
428:	test: 0.8491485	best: 0.8491485 (428)	total: 3m 12s	remaining: 1h 11m 41s
429:	test: 0.8491959	best: 0.8491959 (429)	total: 3m 13s	remaining: 1h 11m 40s
430:	test: 0.8492856	best: 0.8492856 (430)	total: 3m 13s	remaining: 1h 11m 39s
431:	test: 0.8493469	best: 0.8493469 (431)	total: 3m 14s	remaining: 1h 11m 38s
432:	test: 0.8494511	best: 0.8494511 (432)	total: 3m 14s	remaining: 1h 11m 36s
433:	test: 0.8495273	best: 0.8495273 (433)	total: 3m 14s	remaining: 1h 11m 35s
434:	test: 0.8496110	best: 0.8496110 (434)	total: 3m 15s	remaining: 1h 11m 34s
435:	test: 0.8497529	best: 0.8497529 (435)	total: 3m 15s	remaining: 1h 11m 33s
436:	test: 0.8499024	best: 0.8499024 (436)	total: 3m

...

1042:	test: 0.8726447	best: 0.8726447 (1042)	total: 8m 2s	remaining: 1h 8m 59s
1043:	test: 0.8726640	best: 0.8726640 (1043)	total: 8m 2s	remaining: 1h 8m 58s
1044:	test: 0.8726936	best: 0.8726936 (1044)	total: 8m 2s	remaining: 1h 8m 57s
1045:	test: 0.8727222	best: 0.8727222 (1045)	total: 8m 3s	remaining: 1h 8m 57s
1046:	test: 0.8727514	best: 0.8727514 (1046)	total: 8m 3s	remaining: 1h 8m 56s
1047:	test: 0.8727802	best: 0.8727802 (1047)	total: 8m 4s	remaining: 1h 8m 55s
1048:	test: 0.8728187	best: 0.8728187 (1048)	total: 8m 4s	remaining: 1h 8m 55s
1049:	test: 0.8728444	best: 0.8728444 (1049)	total: 8m 5s	remaining: 1h 8m 54s
1050:	test: 0.8728451	best: 0.8728451 (1050)	total: 8m 5s	remaining: 1h 8m 54s
1051:	test: 0.8728606	best: 0.8728606 (1051)	total: 8m 5s	remaining: 1h 8m 53s
1052:	test: 0.8728689	best: 0.8728689 (1052)	total: 8m 6s	remaining: 1h 8m 52s
1053:	test: 0.8728887	best: 0.8728887 (1053)	total: 8m 6s	remaining: 1h 8m 52s
1054:	test: 0.8729165	best: 0.8729165 (1054)	total: 

1145:	test: 0.8752219	best: 0.8752219 (1145)	total: 8m 52s	remaining: 1h 8m 32s
1146:	test: 0.8752615	best: 0.8752615 (1146)	total: 8m 52s	remaining: 1h 8m 32s
1147:	test: 0.8752851	best: 0.8752851 (1147)	total: 8m 53s	remaining: 1h 8m 31s
1148:	test: 0.8753083	best: 0.8753083 (1148)	total: 8m 53s	remaining: 1h 8m 31s
1149:	test: 0.8753360	best: 0.8753360 (1149)	total: 8m 54s	remaining: 1h 8m 30s
1150:	test: 0.8753686	best: 0.8753686 (1150)	total: 8m 54s	remaining: 1h 8m 29s
1151:	test: 0.8753943	best: 0.8753943 (1151)	total: 8m 55s	remaining: 1h 8m 29s
1152:	test: 0.8754137	best: 0.8754137 (1152)	total: 8m 55s	remaining: 1h 8m 28s
1153:	test: 0.8754352	best: 0.8754352 (1153)	total: 8m 55s	remaining: 1h 8m 28s
1154:	test: 0.8754520	best: 0.8754520 (1154)	total: 8m 56s	remaining: 1h 8m 27s
1155:	test: 0.8754755	best: 0.8754755 (1155)	total: 8m 56s	remaining: 1h 8m 26s
1156:	test: 0.8754901	best: 0.8754901 (1156)	total: 8m 57s	remaining: 1h 8m 25s
1157:	test: 0.8754998	best: 0.8754998 (1

1249:	test: 0.8771702	best: 0.8771702 (1249)	total: 9m 40s	remaining: 1h 7m 45s
1250:	test: 0.8771844	best: 0.8771844 (1250)	total: 9m 41s	remaining: 1h 7m 45s
1251:	test: 0.8771816	best: 0.8771844 (1250)	total: 9m 41s	remaining: 1h 7m 45s
1252:	test: 0.8771952	best: 0.8771952 (1252)	total: 9m 42s	remaining: 1h 7m 44s
1253:	test: 0.8772225	best: 0.8772225 (1253)	total: 9m 42s	remaining: 1h 7m 43s
1254:	test: 0.8772577	best: 0.8772577 (1254)	total: 9m 43s	remaining: 1h 7m 43s
1255:	test: 0.8772657	best: 0.8772657 (1255)	total: 9m 43s	remaining: 1h 7m 42s
1256:	test: 0.8772801	best: 0.8772801 (1256)	total: 9m 44s	remaining: 1h 7m 42s
1257:	test: 0.8773086	best: 0.8773086 (1257)	total: 9m 44s	remaining: 1h 7m 42s
1258:	test: 0.8773472	best: 0.8773472 (1258)	total: 9m 44s	remaining: 1h 7m 41s
1259:	test: 0.8773661	best: 0.8773661 (1259)	total: 9m 45s	remaining: 1h 7m 41s
1260:	test: 0.8773919	best: 0.8773919 (1260)	total: 9m 45s	remaining: 1h 7m 40s
1261:	test: 0.8774055	best: 0.8774055 (1

1352:	test: 0.8786444	best: 0.8786444 (1352)	total: 10m 28s	remaining: 1h 6m 57s
1353:	test: 0.8786684	best: 0.8786684 (1353)	total: 10m 29s	remaining: 1h 6m 57s
1354:	test: 0.8786911	best: 0.8786911 (1354)	total: 10m 29s	remaining: 1h 6m 57s
1355:	test: 0.8787112	best: 0.8787112 (1355)	total: 10m 30s	remaining: 1h 6m 56s
1356:	test: 0.8787278	best: 0.8787278 (1356)	total: 10m 30s	remaining: 1h 6m 56s
1357:	test: 0.8787552	best: 0.8787552 (1357)	total: 10m 31s	remaining: 1h 6m 55s
1358:	test: 0.8787800	best: 0.8787800 (1358)	total: 10m 31s	remaining: 1h 6m 55s
1359:	test: 0.8787937	best: 0.8787937 (1359)	total: 10m 32s	remaining: 1h 6m 55s
1360:	test: 0.8788080	best: 0.8788080 (1360)	total: 10m 32s	remaining: 1h 6m 54s
1361:	test: 0.8788219	best: 0.8788219 (1361)	total: 10m 32s	remaining: 1h 6m 54s
1362:	test: 0.8788420	best: 0.8788420 (1362)	total: 10m 33s	remaining: 1h 6m 53s
1363:	test: 0.8788622	best: 0.8788622 (1363)	total: 10m 33s	remaining: 1h 6m 53s
1364:	test: 0.8788858	best: 

1454:	test: 0.8801591	best: 0.8801591 (1454)	total: 11m 19s	remaining: 1h 6m 29s
1455:	test: 0.8801715	best: 0.8801715 (1455)	total: 11m 19s	remaining: 1h 6m 29s
1456:	test: 0.8801875	best: 0.8801875 (1456)	total: 11m 20s	remaining: 1h 6m 28s
1457:	test: 0.8802037	best: 0.8802037 (1457)	total: 11m 20s	remaining: 1h 6m 28s
1458:	test: 0.8802176	best: 0.8802176 (1458)	total: 11m 21s	remaining: 1h 6m 27s
1459:	test: 0.8802245	best: 0.8802245 (1459)	total: 11m 21s	remaining: 1h 6m 26s
1460:	test: 0.8802477	best: 0.8802477 (1460)	total: 11m 22s	remaining: 1h 6m 26s
1461:	test: 0.8802751	best: 0.8802751 (1461)	total: 11m 22s	remaining: 1h 6m 25s
1462:	test: 0.8802945	best: 0.8802945 (1462)	total: 11m 22s	remaining: 1h 6m 25s
1463:	test: 0.8803200	best: 0.8803200 (1463)	total: 11m 23s	remaining: 1h 6m 24s
1464:	test: 0.8803444	best: 0.8803444 (1464)	total: 11m 23s	remaining: 1h 6m 24s
1465:	test: 0.8803528	best: 0.8803528 (1465)	total: 11m 24s	remaining: 1h 6m 23s
1466:	test: 0.8803512	best: 

In [ ]:
# Ensemble
name = 'ensemble'
base_models = [xgboost_model, lightgbm_model, catboost_model]
model_names = ['xgboost', 'lightgbm', 'catboost']
ensemble = Ensemble(lightgbm_stacker_model, base_models, model_names, n_folds=5)
pred = ensemble.fit_predict(X, y, T)

In [14]:
# submission file
sub_df = pd.DataFrame({'ID_code': test_df['ID_code'].values, 'target': pred})
sub_df.to_csv('../submit/{}_submission.csv'.format(name), index=False)